In [1]:
import os
import torch
import torch.nn as nn
import torch.nn.utils.prune as prune
import torch.quantization as tq
from transformers import AutoModelForCausalLM, AutoTokenizer
from torch.profiler import profile, record_function, ProfilerActivity

In [2]:
MODEL_NAME      = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"
PRUNE_AMOUNT    = 0.30      # 30% magnitude pruning
BATCH_SIZE      = 5
MAX_NEW_TOKENS  = 50
PROMPT          = (
    "In a world increasingly driven by artificial intelligence, the ability to interpret "
    "large language models efficiently is crucial for both research and deployment."
)
LOGDIR_BASELINE = "./profiler_logs/baseline"


In [3]:
def load_model(device: torch.device):
    """
    TODO:
      - Load the tokenizer: AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=True)
      - Load the model: AutoModelForCausalLM.from_pretrained(MODEL_NAME, torch_dtype=torch.float32)
      - Move model to `device` and call .eval()
      - Return (tokenizer, model)
    """
    # 1) Load tokenizer
    tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=True)

    # 2) Load FP32 model
    model = AutoModelForCausalLM.from_pretrained(
        MODEL_NAME,
        torch_dtype=torch.float32
    )

    # 3) Move to device and set to eval mode
    model.to(device)
    model.eval()

    # 4) Return both
    return tokenizer, model

In [4]:
def make_batch(tokenizer, prompt: str, device: torch.device):
    """
    TODO:
      - Duplicate `prompt` BATCH_SIZE times into a list of strings
      - Tokenize with padding and truncation: tokenizer(..., return_tensors="pt")
      - Move inputs to `device`
      - Return the tokenized inputs
    """
    # 1) Replicate prompt
    texts = [prompt] * BATCH_SIZE

    # 2) Tokenize with padding & truncation
    inputs = tokenizer(
        texts,
        return_tensors="pt",
        padding=True,
        truncation=True
    )

    # 3) Move to device
    inputs = {k: v.to(device) for k, v in inputs.items()}

    return inputs


In [5]:
def profile_inference(model, inputs, logdir: str, label: str):
    """
    TODO:
      - Create `logdir` if it doesn't exist
      - Use `torch.profiler.profile` (CPU & CUDA, record_shapes, profile_memory, with_stack)
      - Inside the profiler, wrap the call to `model.generate(**inputs, max_new_tokens=MAX_NEW_TOKENS)` in `record_function(label)`
      - After profiling, print:
          1) Top-3 ops by "self_cpu_time_total"
          2) Top-3 ops by "self_cuda_time_total"
          3) Total CPU vs CUDA self-time in milliseconds
      - Note: use `prof.key_averages().table(...)` and sum over `evt.self_cpu_time_total`, `evt.self_cuda_time_total`
      - Traces should be saved automatically by `tensorboard_trace_handler`
    """
    # 1) Ensure log directory exists
    os.makedirs(logdir, exist_ok=True)

    # 2) Run profiler
    with profile(
        activities=[ProfilerActivity.CPU, ProfilerActivity.CUDA],
        record_shapes=True,
        profile_memory=True,
        with_stack=True,
        on_trace_ready=torch.profiler.tensorboard_trace_handler(logdir)
    ) as prof:
        with record_function(label):
            _ = model.generate(**inputs, max_new_tokens=MAX_NEW_TOKENS)

    # 3) Print top‐3 operators by CPU self time
    print(f"\n=== {label} Top-3 ops by CPU self time ===")
    print(prof.key_averages().table(
        sort_by="self_cpu_time_total", row_limit=3
    ))

    # 4) Print top‐3 operators by CUDA self time
    print(f"\n=== {label} Top-3 ops by CUDA self time ===")
    print(prof.key_averages().table(
        sort_by="self_cuda_time_total", row_limit=3
    ))

    # 5) Summarize total CPU vs CUDA self times
    events     = prof.key_averages()
    total_cpu  = sum(evt.self_cpu_time_total for evt in events)
    total_cuda = sum(getattr(evt, "self_cuda_time_total", 0) for evt in events)
    print(f"\n=== {label} Total self-time ===")
    print(f"CPU  : {total_cpu/1e3:.2f} ms")
    print(f"CUDA : {total_cuda/1e3:.2f} ms")

    print(f"\nTrace files for '{label}' written to: {logdir}")

In [6]:
def apply_pruning_and_quant(model: nn.Module):
    """
    TODO:
      - On CPU, apply `prune.l1_unstructured(..., amount=PRUNE_AMOUNT)` to every nn.Linear weight
      - Call `prune.remove(...)` to make masks permanent
      - Then apply `torch.quantization.quantize_dynamic` on {nn.Linear} with dtype=torch.qint8
      - Return the quantized model
    """
    # 1) Prune on CPU
    model.cpu()
    for module in model.modules():
        if isinstance(module, nn.Linear):
            # zero out PRUNE_AMOUNT fraction of the smallest‐magnitude weights
            prune.l1_unstructured(module, name="weight", amount=PRUNE_AMOUNT)
            # make the pruning permanent
            prune.remove(module, "weight")

    # 2) Dynamic 8-bit quantization
    quantized = tq.quantize_dynamic(
        model,
        {nn.Linear},
        dtype=torch.qint8
    )

    return quantized

In [7]:
def start():
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # 1) Load & benchmark baseline
    tokenizer, model = load_model(device)
    batch_inputs     = make_batch(tokenizer, PROMPT, device)
    print("Profiling inference…")
    profile_inference(model, batch_inputs, LOGDIR_BASELINE, label="Baseline")


In [8]:
start()

Profiling baseline inference…


Starting from v4.46, the `logits` model output will have the same type as the model (except at train time, where it will always be FP32)



=== Baseline Top-3 ops by CPU self time ===
-------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                                                   Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg     Self CUDA   Self CUDA %    CUDA total  CUDA time avg       CPU Mem  Self CPU Mem      CUDA Mem  Self CUDA Mem    # of Calls  
-------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                                               Baseline        29.68%     688.039ms       100.00%        2.318s        2.318s       0.000us         0.00%        1.381s        1.381s          